In [0]:
from PIL import Image as image
import numpy as np
import os
import csv

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
enc = OneHotEncoder()

In [0]:
img_rows, img_cols = 256, 256

In [0]:
os.getcwd()

'/content'

# DATA!

In [0]:
path = '/content/drive/My Drive/PlantVillage/'

In [0]:
dirlist = os.listdir(path)

In [0]:
dirlist

['Potato___Late_blight',
 'Pepper__bell___healthy',
 'Pepper__bell___Bacterial_spot',
 'Potato___healthy',
 'Potato___Early_blight',
 'Tomato__Target_Spot',
 'Tomato_Late_blight',
 'Tomato_healthy',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato_Leaf_Mold',
 'Tomato_Early_blight',
 'Tomato_Bacterial_spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus']

In [0]:
folders = os.listdir(path)

records = []

for fname in folders:
    img_paths = os.listdir(path+fname)
    for i_name in img_paths:
        records.append(path+fname+'/'+i_name)

In [0]:
np.random.shuffle(records)

In [0]:
labels = []
for ent in range(len(records)):
    
    label = records[ent].split('/')[5]
    
    labels.append(label)

In [0]:
set(labels[:]), set(labels[1001:2000])

({'Tomato_Late_blight',
  'Tomato_Leaf_Mold',
  'Tomato_Septoria_leaf_spot',
  'Tomato_Spider_mites_Two_spotted_spider_mite',
  'Tomato__Target_Spot',
  'Tomato__Tomato_YellowLeaf__Curl_Virus',
  'Tomato__Tomato_mosaic_virus'},
 {'Tomato_Late_blight',
  'Tomato_Leaf_Mold',
  'Tomato_Septoria_leaf_spot',
  'Tomato_Spider_mites_Two_spotted_spider_mite',
  'Tomato__Target_Spot',
  'Tomato__Tomato_YellowLeaf__Curl_Virus',
  'Tomato__Tomato_mosaic_virus'})

In [0]:
len(records)

10661

In [0]:
train_records = records[ : 8000]
test_records = records[8000 : ]
train_labels = labels[ : 8000]
test_labels = labels[8000 : ]

In [0]:
len(test_records)

2661

In [0]:
len(train_records)

8000

# TEST DATA!

In [0]:
test_pixels = []

test_labels = np.asarray(test_labels)

test_labels = test_labels.reshape(-1, 1)

enc.fit(test_labels)

test_labels = enc.transform(test_labels)

classes = test_labels.shape[1]

for image_name in test_records:
    
    PATH = image_name
    
    # Traversing every Image
    img = image.open(PATH)
        
    width, height = img.size
    mat = img.format
    mode = img.mode

    # Greyscale
    img_grey = img.convert('L')

    # Save as Greyscale 
    value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
    value = value.flatten()
#   value = value.reshape(1, 65536)
    test_pixels.append(value)
        
test_pixels = np.asarray(test_pixels, dtype = 'int')
    
if K.image_data_format() == 'channels_first':
#     train_pixels = train_pixels.reshape(train_pixels.shape[0], 1, img_rows, img_cols)
    test_pixels = test_pixels.reshape(test_pixels.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
#     train_pixels = train_pixels.reshape(train_pixels.shape[0], img_rows, img_cols, 1)
    test_pixels = test_pixels.reshape(test_pixels.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
test_pixels = test_pixels.astype('float32')
    
test_pixels /= 255  

In [0]:
test_pixels.shape

(2661, 256, 256, 1)

In [0]:
test_labels.shape

(2661, 7)

# CNN Architecture!

In [0]:
batch_size = 10
epochs = 12

In [0]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(256, 256, 1), activation='relu'))
model.add(Conv2D(64, (5, 5), input_shape=(256, 256, 1), activation='relu'))

model.add(Conv2D(128, (3, 3), input_shape=(256, 256, 1), activation='relu'))
# model.add(Conv2D(256, (3, 3), input_shape=(256, 256, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(70, activation='relu'))
model.add(Dense(classes, activation='softmax'))
    
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# LOOP!

In [0]:
for count in range(0,5):
    print("loop:"+ str(count))
    train_pixels = []
    
    itr1 = (count*1000)+1
    itr2 = ((count+1)*1000)+1
    
    train_labels = np.asarray(labels[itr1 - 1: itr2 - 1])
    
    train_labels = train_labels.reshape(-1, 1)
    
    train_labels = enc.transform(train_labels)
    
    
    for count1 in range(itr1, itr2):
        
        image_name = train_records[count1]
        PATH = image_name
        
        # Traversing every Image
        img = image.open(PATH)
        
        width, height = img.size
        mat = img.format
        mode = img.mode

        # Greyscale
        img_grey = img.convert('L')

        # Save as Greyscale 
        value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
        value = value.flatten()
#         value = value.reshape(1, 65536)
        train_pixels.append(value)
        
    train_pixels = np.asarray(train_pixels, dtype = 'int')
    
    if K.image_data_format() == 'channels_first':
        train_pixels = train_pixels.reshape(train_pixels.shape[0], 1, img_rows, img_cols)
#         x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        train_pixels = train_pixels.reshape(train_pixels.shape[0], img_rows, img_cols, 1)
#         x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    
    train_pixels = train_pixels.astype('float32')
    
    train_pixels /= 255
    
    
    model.fit(train_pixels, train_labels, batch_size = batch_size, epochs = epochs, verbose=1, validation_data=(test_pixels, test_labels))
    score = model.evaluate(test_pixels, test_labels, verbose=1)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

loop:0
Train on 1000 samples, validate on 2661 samples
Epoch 1/12
1000/1000 [==============================] - 340s 340ms/step - loss: 3.0880 - acc: 0.2540 - val_loss: 1.8496 - val_acc: 0.2946
Epoch 2/12
1000/1000 [==============================] - 330s 330ms/step - loss: 1.8380 - acc: 0.2980 - val_loss: 1.8425 - val_acc: 0.2946
Epoch 3/12
1000/1000 [==============================] - 324s 324ms/step - loss: 1.8133 - acc: 0.3120 - val_loss: 1.8209 - val_acc: 0.2946
Epoch 4/12
1000/1000 [==============================] - 317s 317ms/step - loss: 1.8101 - acc: 0.3120 - val_loss: 1.8167 - val_acc: 0.2946
Epoch 5/12
1000/1000 [==============================] - 325s 325ms/step - loss: 1.8038 - acc: 0.3120 - val_loss: 1.8155 - val_acc: 0.2946
Epoch 6/12
1000/1000 [==============================] - 321s 321ms/step - loss: 1.8130 - acc: 0.3120 - val_loss: 1.8588 - val_acc: 0.2958
Epoch 7/12
 990/1000 [============================>.] - ETA: 2s - loss: 1.7521 - acc: 0.3232

KeyboardInterrupt: ignored

In [0]:
# train_pixels.shape
np.sum(train_labels,axis = 0)

matrix([[130.,  78., 156., 155., 133., 312.,  36.]])

In [0]:
train_labels.shape

(500, 15)

# TIMEPASS!

In [0]:
path = '/home/vilkhu/Sabudh!/PAU/plant_disease/Plant_village'

In [0]:
dircs = sorted(os.listdir(path))

pixels = []

In [0]:
def img_to_pixel(img):
    
    img = image.open(PATH + '/' + img)
    width, height = img.size
    mat = img.format
    mode = img.mode

    img_grey = img.convert('L')

    value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
    value = value.flatten()
#     value = value.reshape(1,65536)
        
    yield value

In [0]:
with open('data.csv', 'w', newline = '') as file:

    for dirc in dircs:
    
        PATH = path + '/' + dirc
        dirc_list = sorted(os.listdir(PATH))   

    
        for img in dirc_list:
            for itr in img_to_pixel(img):
            
                writer = csv.writer(file)
                writer.writerow(itr)

In [0]:
import pandas as pd
import csv

In [0]:
pdata = pd.read_csv('data.csv')

In [0]:
labs=[{"name":'Pepper_bell_Bacterial_spot',"times":997},
    {"name":'Pepper_bell_healthy',"times":1478},
    {"name":'Potato_Early_blight',"times":1000},
    {"name":'Potato_healthy',"times":152},
    {"name":'Potato_Late_blight',"times":1000},
    {"name":'Tomato_Bacterial_spot',"times":2127},
    {"name":'Tomato_Early_blight',"times":1000},
    {"name":'Tomato_healthy',"times":1591},
    {"name":'Tomato_Late_blight',"times":1909},
    {"name":'Tomato_Leaf_Mold',"times":952},
    {"name":'Tomato_Septoria_leaf_spot',"times":1771},
    {"name":'Tomato_Spider_mites_Two_spotted_spider_mite',"times":1676},
    {"name":'Tomato_Target_Spot',"times":1404},
    {"name":'Tomato_Tomato_mosaic_virus',"times":373},
    {"name":'Tomato_Tomato_YellowLeaf_Curl_Virus',"times":3208}]

labels=[]
li=[]
for i in labs:
    li=[i["name"]]*i['times']
    labels.extend(li)

# ENDS HERE!